# [Construct Binary Tree from Preorder and Inorder Traversal](https://leetcode.com/problems/construct-binary-tree-from-preorder-and-inorder-traversal/)
## Description
Build a binary tree from preorder and inorder traversal arrays.
## Strategy
Use recursion and map of inorder indexes. Reconstruct root from preorder and partition in inorder.
## Walkthrough
**Given:**
- `preorder = [3, 9, 20, 15, 7]`
- `inorder = [9, 3, 15, 20, 7]`

**Goal:** Reconstruct the original binary tree

## Step-by-Step Construction

### Step 1: Identify Root
**Rule:** First element in preorder is always the root

```
preorder = [3, 9, 20, 15, 7]
            ↑
         Root = 3
```

**Create root node:**
```
    3
```

### Step 2: Split Inorder Array
**Rule:** Find root in inorder to separate left and right subtrees

```
inorder = [9, 3, 15, 20, 7]
           ↑  ↑  ↑
        Left  Root  Right

Left subtree inorder: [9]
Right subtree inorder: [15, 20, 7]
```

### Step 3: Split Preorder Array
**Rule:** Use inorder subtree sizes to split preorder

```
Left subtree size = 1 (just node 9)
Right subtree size = 3 (nodes 15, 20, 7)

preorder = [3, 9, 20, 15, 7]
            ↑  ↑  ↑
         Root Left Right

Left subtree preorder: [9]
Right subtree preorder: [20, 15, 7]
```

### Step 4: Recursively Build Left Subtree
**Input:** 
- Left preorder: `[9]`
- Left inorder: `[9]`

**Process:**
1. Root = 9 (first in preorder)
2. Find 9 in inorder: `[] | 9 | []` (no left or right)
3. Create leaf node

```
    3
   /
  9
```

### Step 5: Recursively Build Right Subtree
**Input:**
- Right preorder: `[20, 15, 7]`
- Right inorder: `[15, 20, 7]`

**Process:**
1. Root = 20 (first in preorder)
2. Find 20 in inorder: `[15] | 20 | [7]`

```
    3
   / \
  9   20
```

### Step 6: Build Right Subtree's Left Child
**Input:**
- Left preorder: `[15]`
- Left inorder: `[15]`

**Process:**
1. Root = 15 (single node)
2. No children

```
    3
   / \
  9   20
     /
    15
```

### Step 7: Build Right Subtree's Right Child
**Input:**
- Right preorder: `[7]`
- Right inorder: `[7]`

**Process:**
1. Root = 7 (single node)
2. No children

```
    3
   / \
  9   20
     / \
    15  7
```

## Final Result

**Constructed Tree:**
```
       3
      / \
     9   20
        / \
       15  7
```

**Verification:**
- Preorder traversal: 3 → 9 → 20 → 15 → 7 ✅
- Inorder traversal: 9 → 3 → 15 → 20 → 7 ✅


In [ ]:
import { TreeNode } from "../bin/utils.ts";

export function buildTree(preorder: number[], inorder: number[]): TreeNode | null {
  if (!preorder.length || !inorder.length) return null

  const rootValue: number = preorder[0]
  const root = new TreeNode(rootValue)

  // find root in inorder
  const rootIndex: number = inorder.findIndex(val => val === rootValue)

  // split the left and right pre/inorder arrays
  const lInorder: number[] = inorder.slice(0, rootIndex)
  const rInorder: number[] = inorder.slice(rootIndex + 1, inorder.length)
  const lPreorder: number[] = preorder.slice(1, 1 + lInorder.length)
  const rPreorder: number[] = preorder.slice(1 + rInorder.length)

  // recurse on the children
  root.left = buildTree(lPreorder, lInorder)
  root.right = buildTree(rPreorder, rInorder)

  return root;
}


In [3]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("Build Tree - Basic", () => {
  const res = buildTree([3,9,20,15,7], [9,3,15,20,7]);
  assertEquals(res?.val, 3);
});



Build Tree - Basic ... FAILED (2ms)

 ERRORS 

Build Tree - Basic => <anonymous>:2:6
error: AssertionError: Values are not equal.


    [Diff] Actual / Expected


-   undefined
+   3

  throw new AssertionError(message);
        ^
    at Module.assertEquals (https://deno.land/std@0.224.0/assert/assert_equals.ts:52:9)
    at assertEquals (https://deno.land/std@0.224.0/testing/asserts.ts:101:11)
    at <anonymous>:16:3

 FAILURES 

Build Tree - Basic => <anonymous>:2:6

FAILED | 0 passed | 1 failed (2ms)
